In [1]:
import configparser
import os
import time
import ftplib
import datetime
import shutil

In [2]:
def ftpconnstring():
    config = configparser.ConfigParser()
    config.read('config.INI')
    ftphost = config['FTPCONN']['host']
    ftpusr = config['FTPCONN']['username']
    ftppwd = config['FTPCONN']['password']

    ftp_server = ftplib.FTP(ftphost, ftpusr, ftppwd, timeout=60)
    ftp_server.encoding = "utf-8"
    ftp_server.set_pasv(True)
    ftp_server.connect()
    ftp_server.login(ftpusr, ftppwd)
    return ftp_server

In [3]:
def now():
    dtnow = datetime.datetime.now()
    return dtnow

In [ ]:
while True:   
    print(now().strftime("%Y-%m-%d %H:%M:%S"), 'Checking Database Backup Folder Process') 
    config = configparser.ConfigParser()
    config.read('config.INI')
    # Backup Database Process
    src = config['PATH']['source']
    dst = config['PATH']['dest']
    file = os.path.exists(src)
    sumfile = os.listdir(src)
    if file == True:
        if len(sumfile) <= 4:
            print(now().strftime("%Y-%m-%d %H:%M:%S"), "Running Backup Process")
            ftpconnstring().connect()
            for x in sumfile:
                print(now().strftime("%Y-%m-%d %H:%M:%S"), x)
                y = os.path.join(src, x)
                #if os.path.isfile(src):
                with open(y, "rb") as file:
                    # define dest folder and the file from src to be copied
                    ftpconnstring().storbinary(f'STOR {dst}' + os.path.basename(x), file)
            ftpconnstring().quit()
        else:
            print(now().strftime("%Y-%m-%d %H:%M:%S"), "Please Check MonitorDB Service")
    else:
        print(now().strftime("%Y-%m-%d %H:%M:%S"), "Please SQL Agent Service")
    
    #Backup SuratJalan Process
    print(now().strftime("%Y-%m-%d %H:%M:%S"), 'Checking SuratJalan Folder Process')
    config = configparser.ConfigParser()
    config.read('config.INI')
    srcSJ = config['PATH']['sourceSJ']
    dstSJ = config['PATH']['destSJ']
    fileSJ = os.path.exists(srcSJ)
    sumfileSJ = os.listdir(srcSJ)
    newlytime = os.path.getctime(srcSJ)
    newlydate = datetime.datetime.fromtimestamp(newlytime)
    latestsrc = newlydate.strftime('%Y%m%d')
    last_updater = open('lastUpdated.txt', 'r')
    latest = last_updater.read()
    if latestsrc == latest:
        print(now().strftime("%Y-%m-%d %H:%M:%S"),'Already Updated')
    elif latestsrc > latest:
        print(now().strftime("%Y-%m-%d %H:%M:%S"), 'Syncronize Process')
        ftpconnstring().connect()
        c = []
        for x in sumfileSJ:
            checkstartpoint = x > latest
            if checkstartpoint == True:
                y = os.path.join(srcSJ, x+'\\')
                c.append(y)
        for v in range(len(c)):
            b = os.listdir(c[v])
            a = c[v].split('\\\\')
            p = a[4].split('\\')
            f = dstSJ+'/'+p[0]+'/'
            namefile = os.path.exists(f)
            try:
                if namefile == False:
                    q = ftpconnstring().mkd(f)
                    print(now().strftime("%Y-%m-%d %H:%M:%S"), 'Create New Directory ',q)
            except ftplib.error_perm as e:
                print(now().strftime("%Y-%m-%d %H:%M:%S"),f'{e}')
            for r in range(len(b)):
                k = os.path.join(c[v], b[r])
                print(now().strftime("%Y-%m-%d %H:%M:%S"), 'Move File from: ',k)
                print(now().strftime("%Y-%m-%d %H:%M:%S"), 'Move File to: ',f)
                ftpconnstring().connect()  
                with open(k, "rb") as fileSJ:
                # define dest folder and the file from src to be copied
                    ftpconnstring().storbinary(f'STOR {f}' + os.path.basename(k), fileSJ)     
                ftpconnstring().quit()  
    else:
        print(now().strftime("%Y-%m-%d %H:%M:%S"), 'No Need Action') 

    #Deleting Old SuratJalan Folder
    print(now().strftime("%Y-%m-%d %H:%M:%S"), 'Calculate SuratJalan Folder')
    checkdt = datetime.datetime.now()
    checkdtformat = checkdt.strftime('%Y')
    config = configparser.ConfigParser()
    config.read('config.INI')
    srcSJ = config['PATH']['sourceSJ']
    dstSJ = config['PATH']['destSJ']
    sumfileSJ = os.listdir(srcSJ)
    dateSJ = os.path.getctime(srcSJ)
    dateSJformat = datetime.datetime.fromtimestamp(dateSJ)
    yearonly = dateSJformat.strftime('%Y')
    periodSJ = int(checkdtformat) - int(yearonly)
    c = []
    for x in sumfileSJ:
            checkstartpoint = x > latest
            if checkstartpoint == True:
                y = os.path.join(srcSJ, x+'\\')
                c.append(y)
    for v in range(len(c)):
        b = os.listdir(c[v])
        a = c[v].split('\\\\')
        p = a[4].split('\\')
        f = dstSJ+'/'+p[0]+'/'
    for sj in os.listdir(srcSJ):
        print(now().strftime("%Y-%m-%d %H:%M:%S"), 'Checking Source Folder') 
        if periodSJ >= 1:
            print(now().strftime("%Y-%m-%d %H:%M:%S"), sj ,"Proses Delete Old SuratJalan")
            print(now().strftime("%Y-%m-%d %H:%M:%S"), 'delete: ', c[v])
            shutil.rmtree(c[v])
        else:
            print(now().strftime("%Y-%m-%d %H:%M:%S"), sj, 'No Need Action')
    
    last_update = open('lastUpdated.txt', 'r+')
    last_update.write(latestsrc)
    last_update.close()

    delay = config['DELAY']['sleep']
    print(now().strftime("%Y-%m-%d %H:%M:%S"), "Waiting next " + delay + " Hours")
    time.sleep(int(delay) * 3600)


In [6]:
checkdt = datetime.datetime.now()
checkdtformat = checkdt.strftime('%Y')
config = configparser.ConfigParser()
config.read('config.INI')
srcSJ = config['PATH']['sourceSJ']
dstSJ = config['PATH']['destSJ']
dateSJ = os.path.getctime(srcSJ)
sumfileSJ = os.listdir(srcSJ)
dateSJformat = datetime.datetime.fromtimestamp(dateSJ)
newlytime = os.path.getctime(srcSJ)
newlydate = datetime.datetime.fromtimestamp(newlytime)
latestsrc = newlydate.strftime('%Y%m%d')
last_updater = open('lastUpdated.txt', 'r')
latest = last_updater.read()
yearonly = dateSJformat.strftime('%Y')
periodSJ = int(checkdtformat) - int(yearonly)
c = []
for x in sumfileSJ:
    checkstartpoint = x < latest
    if checkstartpoint == False:
       print('Still Meet Exception')   
    else:
        y = os.path.join(srcSJ, x+'\\')
        c.append(y)
        for v in range(len(c)):
            b = os.listdir(c[v])
            a = c[v].split('\\\\')
            p = a[4].split('\\')
            f = dstSJ+'/'+p[0]+'/'
            print(c[v])
    for sj in os.listdir(srcSJ):
        print(now().strftime("%Y-%m-%d %H:%M:%S"), 'Checking Source Folder') 
        if periodSJ >= 1:
            print(now().strftime("%Y-%m-%d %H:%M:%S"), sj ,"Proses Delete Old SuratJalan")
            print(now().strftime("%Y-%m-%d %H:%M:%S"), 'delete: ', c[v])
            #shutil.rmtree(c[v])
        else:
            print(now().strftime("%Y-%m-%d %H:%M:%S"), sj, 'No Need Action')


C:\\MPV_Production\\HostCommSJ\\SuratJalanMPV\\20180123\
2023-04-28 11:53:15 Checking Source Folder
2023-04-28 11:53:15 20180123 No Need Action
2023-04-28 11:53:15 Checking Source Folder
2023-04-28 11:53:15 20190123 No Need Action
2023-04-28 11:53:15 Checking Source Folder
2023-04-28 11:53:15 20230331 No Need Action
C:\\MPV_Production\\HostCommSJ\\SuratJalanMPV\\20180123\
C:\\MPV_Production\\HostCommSJ\\SuratJalanMPV\\20190123\
2023-04-28 11:53:15 Checking Source Folder
2023-04-28 11:53:15 20180123 No Need Action
2023-04-28 11:53:15 Checking Source Folder
2023-04-28 11:53:15 20190123 No Need Action
2023-04-28 11:53:15 Checking Source Folder
2023-04-28 11:53:15 20230331 No Need Action
Still Meet Exception
2023-04-28 11:53:15 Checking Source Folder
2023-04-28 11:53:15 20180123 No Need Action
2023-04-28 11:53:15 Checking Source Folder
2023-04-28 11:53:15 20190123 No Need Action
2023-04-28 11:53:15 Checking Source Folder
2023-04-28 11:53:15 20230331 No Need Action


In [ ]:
for sj in os.listdir(srcSJ):
    print(now().strftime("%Y-%m-%d %H:%M:%S"), 'Checking Source Folder') 
    if periodSJ == 0:
        print(now().strftime("%Y-%m-%d %H:%M:%S"), "Proses Delete Old SuratJalan")
        print(now().strftime("%Y-%m-%d %H:%M:%S"), 'delete: ', c[v])
        shutil.rmtree(c[v])
    else:
        print(now().strftime("%Y-%m-%d %H:%M:%S"), sj, 'No Need Action')


In [ ]:
print(periodSJ)